In [ ]:
# ### Cell 1: Install dependencies (GPU + HD-BET)
!pip install hd-bet
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
!pip install nibabel pandas matplotlib scipy


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.2 MB/s eta 

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [ ]:
# ### Cell 2: Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [5]:
!unzip /content/drive/MyDrive/ADNI/Half.zip

Streaming output truncated to the last 5000 lines.
   creating: Half/016_S_0991/MPR__GradWarp__B1_Correction__N3__Scaled/2007-12-13_14_26_09.0/I88024/
  inflating: __MACOSX/Half/016_S_0991/MPR__GradWarp__B1_Correction__N3__Scaled/2007-12-13_14_26_09.0/._I88024  
   creating: Half/016_S_0991/MPR__GradWarp__B1_Correction__N3__Scaled/2007-07-30_14_55_29.0/I74222/
  inflating: __MACOSX/Half/016_S_0991/MPR__GradWarp__B1_Correction__N3__Scaled/2007-07-30_14_55_29.0/._I74222  
   creating: Half/016_S_0354/MPR__GradWarp__B1_Correction__N3__Scaled/2007-07-11_13_15_37.0/I101389/
  inflating: __MACOSX/Half/016_S_0354/MPR__GradWarp__B1_Correction__N3__Scaled/2007-07-11_13_15_37.0/._I101389  
   creating: Half/016_S_0354/MPR__GradWarp__B1_Correction__N3__Scaled/2006-05-05_10_42_59.0/I92669/
  inflating: __MACOSX/Half/016_S_0354/MPR__GradWarp__B1_Correction__N3__Scaled/2006-05-05_10_42_59.0/._I92669  
   creating: Half/016_S_0354/MPR__GradWarp__B1_Correction__N3__Scaled/2006-12-06_12_47_58.0/I86040/

In [ ]:
### Cell 3: Define paths & imports
import os, subprocess, gc
import torch, numpy as np, pandas as pd, nibabel as nib
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from concurrent.futures import ProcessPoolExecutor

# === CONFIGURATION ===
ROOT_DIR   = "/content/Half"
CSV_PATH   = "/content/drive/MyDrive/ADNI/ADNI1_Baseline_Only.csv"
OUTPUT_DIR = "/content/drive/MyDrive/ADNI/Processed_224x224"
os.makedirs(OUTPUT_DIR, exist_ok=True)

HD_BET_BIN   = "hd-bet"
NUM_SLICES   = 50
BRAIN_THRESH = 1e-6

# Use exactly "cuda" so HD-BET will load its model on your GPU
DEVICE_IDX = "cuda" if torch.cuda.is_available() else "cpu"
print(f"→ GPU enabled? {DEVICE_IDX=='cuda'} (using device: {DEVICE_IDX})")


def run_hdbet(in_nii: str, out_base: str) -> str:
    """
    Run HD-BET to produce skull-stripped NIfTI.
    Prints stderr if it fails.
    """
    os.makedirs(os.path.dirname(out_base), exist_ok=True)
    out_file = out_base + ".nii.gz"
    cmd = [
        HD_BET_BIN,
        "-i", in_nii,
        "-o", out_file,
        "-device", DEVICE_IDX,
        "--disable_tta"  # no test-time augment
    ]
    print("    ↪", " ".join(cmd))
    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("    ⚠️ HD-BET failed:")
        print("      stderr:", e.stderr.strip())
        raise
    if not os.path.exists(out_file):
        raise RuntimeError(f"Expected stripped file missing: {out_file}")
    return out_file


def save_slice_png(args):
    """
    Worker to resize and save a single slice.
    """
    slice_array, path = args
    zf = (224/slice_array.shape[0], 224/slice_array.shape[1])
    sl_r = zoom(slice_array, zf, order=1)
    plt.imsave(path, sl_r, cmap="gray")


def extract_sagittal_slices(stripped_nii: str, output_dir: str):
    """
    Gather valid sagittal slices, then dispatch
    the PNG‐saving into a small process pool.
    """
    os.makedirs(output_dir, exist_ok=True)
    img  = nib.load(stripped_nii)
    data = img.get_fdata().astype(np.float32)
    mn, mx = data.min(), data.max()
    data = (data - mn) / (mx - mn + 1e-8)
    data[data < BRAIN_THRESH] = 0

    valid = [data[i] for i in range(data.shape[0]) if data[i].max() > BRAIN_THRESH]
    if not valid:
        print("      ⚠️ No valid slices found")
        return

    step   = max(1, len(valid)//NUM_SLICES)
    chosen = valid[::step][:NUM_SLICES]

    # Build arguments for each PNG
    tasks = []
    for idx, sl in enumerate(chosen):
        out_path = os.path.join(output_dir, f"sagittal_{idx:03d}.png")
        tasks.append((sl, out_path))

    # Parallelize the relatively cheap resizing+IO
    with ProcessPoolExecutor(max_workers=4) as pool:
        pool.map(save_slice_png, tasks)

    gc.collect()


### Cell 5: Main loop (skip if already done)
df   = pd.read_csv(CSV_PATH)
seen = set()

for subj in df["PTID"].unique():
    if subj in seen:
        continue

    # where we’d save slices
    png_dir = os.path.join(OUTPUT_DIR, subj, "sagittal_slices")

    # skip if already have 50 or more PNGs
    if os.path.isdir(png_dir):
        existing = [f for f in os.listdir(png_dir) if f.startswith("sagittal_") and f.endswith(".png")]
        if len(existing) >= NUM_SLICES:
            print(f"[{subj}] ▶️ already has {len(existing)} slices, skipping")
            seen.add(subj)
            continue

    subj_dir = os.path.join(ROOT_DIR, subj)
    if not os.path.isdir(subj_dir):
        print(f"[{subj}] ⚠️ dir-not-found")
        continue

    # find first Scaled .nii
    nii_path = None
    for r, _, files in os.walk(subj_dir):
        for f in files:
            if "Scaled" in f and f.endswith((".nii", ".nii.gz")):
                nii_path = os.path.join(r, f)
                break
        if nii_path:
            break

    if not nii_path:
        print(f"[{subj}] ⛔ no Scaled .nii")
        continue

    print(f"[{subj}] processing {os.path.basename(nii_path)}")
    try:
        outdir = os.path.join(os.path.dirname(nii_path), "hdbet_output")
        os.makedirs(outdir, exist_ok=True)
        base   = os.path.join(outdir, "brain_stripped")

        print("    → Running HD-BET")
        stripped = run_hdbet(nii_path, base)

        print("    → Extracting & saving slices")
        extract_sagittal_slices(stripped, png_dir)

        seen.add(subj)
        print(f"[{subj}] ✅ done\n")

    except Exception as e:
        print(f"[{subj}] ❌ failed: {e}\n")
        gc.collect()



→ GPU enabled? True (using device: cuda)
[011_S_0002] ⚠️ dir-not-found
[011_S_0003] ▶️ already has 50 slices, skipping
[022_S_0004] ▶️ already has 50 slices, skipping
[011_S_0005] ▶️ already has 50 slices, skipping
[100_S_0006] ⚠️ dir-not-found
[022_S_0007] ⚠️ dir-not-found
[011_S_0010] ▶️ already has 50 slices, skipping
[022_S_0014] ▶️ already has 50 slices, skipping
[100_S_0015] ⚠️ dir-not-found
[011_S_0016] ▶️ already has 50 slices, skipping
[011_S_0021] ▶️ already has 50 slices, skipping
[011_S_0023] ▶️ already has 50 slices, skipping
[067_S_0029] ⚠️ dir-not-found
[100_S_0035] ⚠️ dir-not-found
[067_S_0038] ⚠️ dir-not-found
[099_S_0040] ⚠️ dir-not-found
[023_S_0042] ▶️ already has 50 slices, skipping
[018_S_0043] ▶️ already has 50 slices, skipping
[067_S_0045] ⚠️ dir-not-found
[100_S_0047] ⚠️ dir-not-found
[099_S_0051] ⚠️ dir-not-found
[011_S_0053] ▶️ already has 50 slices, skipping
[099_S_0054] ⚠️ dir-not-found
[067_S_0056] ⚠️ dir-not-found
[018_S_0057] ▶️ already has 50 slices, sk